<a href="https://colab.research.google.com/github/ogidanayodeji/implementation-of-cryptographic-algorithms-for-encryption-and-decryption-/blob/main/github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install cryptography
import hashlib
from cryptography.hazmat.primitives import hashes, hmac
from cryptography.hazmat.primitives.asymmetric import rsa, padding, utils
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.exceptions import InvalidSignature
import os
from google.colab import files
uploaded = files.upload()

Saving cryptography.txt to cryptography.txt


In [4]:
#Encryption
def generate_rsa_keypair():
    # Generate RSA key pair: a private key and its corresponding public key
    private_key = rsa.generate_private_key(
        public_exponent=65537,  # Public exponent value
        key_size=2048,  # Key size (in bits) for RSA keys
        backend=default_backend()  # Backend cryptographic provider
    )
    public_key = private_key.public_key()  # Derive the public key
    return private_key, public_key

def encrypt_file_aes(file_path, aes_key):
    # Encrypts a file using AES algorithm
    with open(file_path, 'rb') as f:  # Open the file to encrypt it
        data = f.read()  # Read the file data

    iv = os.urandom(16)  # Generate a random 128-bit Initialization Vector (IV) for AES

    # Initialize the AES Cipher in GCM mode with the given AES key and IV
    cipher = Cipher(algorithms.AES(aes_key), modes.GCM(iv), backend=default_backend())
    encryptor = cipher.encryptor()  # Create an encryptor object
    encrypted_data = encryptor.update(data) + encryptor.finalize()  # Encrypt the data

    return encrypted_data, encryptor.tag, iv  # Return the encrypted data, tag, and IV

def encrypt_key_with_rsa(aes_key, public_key):
    # Encrypt the AES key using the RSA public key
    encrypted_key = public_key.encrypt(
        aes_key,  # AES key to be encrypted
        padding.OAEP(  # Optimal Asymmetric Encryption Padding (OAEP)
            mgf=padding.MGF1(algorithm=hashes.SHA256()),  # Mask Generation Function
            algorithm=hashes.SHA256(),  # Hash algorithm
            label=None  # Optional label
        )
    )
    return encrypted_key  # Return the encrypted AES key

def create_hmac(data, key):
    # Create HMAC (Hash-based Message Authentication Code) for data integrity
    h = hmac.HMAC(key, hashes.SHA256(), backend=default_backend())  # Initialize HMAC object
    h.update(data)  # Update HMAC with the data
    return h.finalize()  # Finalize HMAC and return

def decrypt_file_aes(encrypted_data, aes_key, tag, iv):
    # Decrypts a file encrypted by AES algorithm
    # Initialize the AES Cipher in GCM mode with the given AES key, IV, and tag
    cipher = Cipher(algorithms.AES(aes_key), modes.GCM(iv, tag), backend=default_backend())
    decryptor = cipher.decryptor()  # Create a decryptor object
    decrypted_data = decryptor.update(encrypted_data) + decryptor.finalize()  # Decrypt the data

    return decrypted_data  # Return the decrypted data

def decrypt_key_with_rsa(encrypted_key, private_key):
    # Decrypt the AES key using the RSA private key
    aes_key = private_key.decrypt(
        encrypted_key,  # Encrypted AES key
        padding.OAEP(  # OAEP padding
            mgf=padding.MGF1(algorithm=hashes.SHA256()),  # Mask Generation Function
            algorithm=hashes.SHA256(),  # Hash algorithm
            label=None  # Optional label
        )
    )
    return aes_key  # Return the decrypted AES key

def verify_hmac(data, key, hmac_to_check):
    # Verify the HMAC for data integrity
    h = hmac.HMAC(key, hashes.SHA256(), backend=default_backend())  # Initialize HMAC object
    h.update(data)  # Update HMAC with the data
    try:
        h.verify(hmac_to_check)  # Attempt to verify the HMAC
        return True  # Return True if verification is successful
    except InvalidSignature:
        return False  # Return False if verification fails

        #digital signature
def generate_file_checksum(file_path):
    sha256_hash = hashlib.sha256()
    with open(file_path, "rb") as f:
        for byte_block in iter(lambda: f.read(4096), b""):
            sha256_hash.update(byte_block)
    return sha256_hash.hexdigest()

def sign_data(private_key, data):
    digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
    digest.update(data)
    hashed_data = digest.finalize()
    signature = private_key.sign(
        hashed_data,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        utils.Prehashed(hashes.SHA256())
    )
    return signature

def verify_signature(public_key, signature, data):
    # Initialize a SHA-256 hash object
    digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
    # Update the hash object with the original data
    digest.update(data)
    # Finalize the hash to get the digest
    hashed_data = digest.finalize()

    try:
        # Use the RSA public key to verify the signature against the hashed data
        public_key.verify(
            signature,          # The signature that was created with the private key
            hashed_data,        # The hashed data that the signature is supposed to match
            padding.PSS(        # The padding scheme used when signing
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            utils.Prehashed(hashes.SHA256())  # Indicates that the data is already hashed
        )
        # If verify() does not raise an exception, the signature is valid
        return True
    except InvalidSignature:
        # If verify() raises an InvalidSignature exception, the signature is invalid
        return False


                 #Encryption main script
# Generate RSA key pairs for two users
private_key_user1, public_key_user1 = generate_rsa_keypair()
private_key_user2, public_key_user2 = generate_rsa_keypair()

# Generate a random 256-bit AES key
aes_key = os.urandom(32)

# Encrypt the file with AES and get the encrypted data, tag, and IV
encrypted_data, tag, iv = encrypt_file_aes('cryptography.txt', aes_key)

# Create an HMAC for the encrypted data using the AES key
hmac_value = create_hmac(encrypted_data, aes_key)

# Encrypt the AES key with the RSA public keys of both users
encrypted_aes_key_user1 = encrypt_key_with_rsa(aes_key, public_key_user1)
encrypted_aes_key_user2 = encrypt_key_with_rsa(aes_key, public_key_user2)

# Save encrypted data, tag, IV, HMAC, and the encrypted AES keys to files
with open('encrypted_cryptography.txt', 'wb') as f:
    f.write(encrypted_data)
# Save the rest of the data (tag, iv, hmac_value, encrypted keys) accordingly


                  #decryption main script

# Assuming the private key of User1 is loaded from a secure storage
# private_key_user1 = ...

# Read the necessary data from files
with open('encrypted_cryptography.txt', 'rb') as f:
    encrypted_data = f.read()
# Load the rest of the data (tag, iv, hmac_value, encrypted keys) accordingly

# Decrypt the AES key using the RSA private key of User1
decrypted_aes_key_user1 = decrypt_key_with_rsa(encrypted_aes_key_user1, private_key_user1)

# Decrypt the file data using the decrypted AES key
decrypted_data = decrypt_file_aes(encrypted_data, decrypted_aes_key_user1, tag, iv)

# Verify the integrity of the decrypted data using HMAC
is_intact = verify_hmac(encrypted_data, decrypted_aes_key_user1, hmac_value)

# Read the original data from the file for comparison after decryption
original_data = open('cryptography.txt', 'rb').read()

# Save decrypted data to a file
with open('decrypted_cryptography.txt', 'wb') as f:
    f.write(decrypted_data)

                     # signature script execution

private_key_user1, public_key_user1 = generate_rsa_keypair()

# Assuming the file 'cryptography.txt' exists and contains data to be signed
file_path = 'cryptography.txt'
with open(file_path, 'rb') as f:
    file_data = f.read()

original_checksum = generate_file_checksum('cryptography.txt')
decrypted_checksum = generate_file_checksum('decrypted_cryptography.txt')
signature = sign_data(private_key_user1, file_data)
signature_valid = verify_signature(public_key_user1, signature, file_data)

                             #Encryption output

# After saving all the necessary components, print the success message
print("Encryption process completed successfully.")
print("The file has been encrypted and the AES key has been secured with RSA.")

                           #decryption output
# run an integrity check
# Output results
if original_data == decrypted_data:
  print("Integrity check passed")
  print("Decryption process completed successfully.")

else:
    print("Decryption process completed, but the integrity check failed.")
    print("The data may have been tampered with or the wrong key was used.")

                                   #signature output

#print("Checksum of the file:", original_checksum)
print("Original and decrypted checksum match:", original_checksum == decrypted_checksum)
print("Signature valid:", signature_valid)


Encryption process completed successfully.
The file has been encrypted and the AES key has been secured with RSA.
Integrity check passed
Decryption process completed successfully.
Original and decrypted checksum match: True
Signature valid: True
